## Introdução

Feature engineering é o processo no qual é possível transformar, extrair e criar novas features a partir dos dados disponíveis, com o objetivo de melhorar o desempenho de algoritmos de aprendizado de máquina. Isso faz com que seja essencial conhecer essas técnicas, para aqueles que pretendem participar em competições de ciência de dados.

Como será mostrado nesse notebook, esse procedimento pode aumentar significamente a precisão até mesmo dos modelos mais simples. O objetivo desse projeto será gerar um modelo para predizer a temperatura máxima em um dia qualquer no futuro. 

Porém, só será utilizada uma feature fornecida nos dados: a data na qual a temperatura foi registrada. Além disso, o modelo utilizado será uma regressão linear simples ou com regularização. A data foi escolhida, pois muitas competições utilizam séries temporais, logo as técnicas abordadas nesse material poderão ser úteis em uma variedade de projetos.

## 0 - Importando os módulos necessários

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
%matplotlib inline

## 1 - Examinando os dados
Nesse notebook, será utilizado o dataset de clima da Austrália, que possui 10 anos de observações diárias do clima em diversas regiões do país.

Link do dataset: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [ ]:
# Lendo os dados
df = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

In [ ]:
# Informações básicas sobre o conjunto de dados
print('Formato do DataFrame: ', df.shape)
print('Número de registros: ', df.shape[0], '\n')
print('Localizações: ', df['Location'].unique())

In [ ]:
#Informações sobre valores nulos e tipos de dados
print(df.info())

Para simplificar a análise, vamos utilizar apenas a coluna que representa a temperatura máxima (MaxTemp) em Sydney, realizando regressões tentar capturar as tendências dessa feature em relação ao tempo.

In [ ]:
# Convertendo 'Date' para DateTime
df['Date'] = pd.to_datetime(df['Date'])

# Selecionando Sydney
df_sidney = df[df['Location'] == 'Sydney']

# Definindo a data como index e selecionando apenas a coluna MaxTemp
# Note que [["MaxTemp"]] foi usado para obter um DataFrame. Se ["MaxTemp"]
# fosse utilizado, a operacao retornaria um objeto Series do pandas.
df_sidney = df_sidney.set_index('Date')[["MaxTemp"]].dropna().sort_index()

print(df_sidney.head(), '\n')
print(type(df_sidney))
print(df_sidney.shape)

In [ ]:
df_sidney.plot(figsize=(20, 10))

Nota-se que a temperatura possui uma relação forte com a data. Portanto, vamos extrair features a partir da data para tentar prever a temperatura em datas futuras.

Antes disso, precisamos separar um conjunto de validação, como estamos tentando prever o clima em datas futuras, não podemos extrair observações em datas aleatórias.
Para obter um conjunto de validação robusto, vamos treinar o model com observações até o final de 2015. Os dados a partir de 2016 serão utilizados para validação.

## 2 - Tratando os dados e preparando funções

In [ ]:
# Separando treino e validação
train_df = df_sidney[:'2015-12-31']
valid_df = df_sidney['2016-01-01':]

fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(train_df.index, train_df['MaxTemp'], color='b', label='Treino')
ax.plot(valid_df.index, valid_df['MaxTemp'], color='r', label='Validação')
ax.legend()
ax.set_title('Divisão entre treino e validação')
ax.set_xlabel('Data')
ax.set_ylabel('Temperatura Máxima (MaxTemp)')

In [ ]:
# Resetando os índices para extrair features da data
train_df.reset_index(inplace=True)
valid_df.reset_index(inplace=True)

In [ ]:
def mostrar_predicoes(train_df_features, valid_df_features, model = None):
    """
    Recebe os DataFrames de treino e validacao, treina o modelo,
    e gera uma visualizacao grafica das predicoes para esses conjuntos de dados.
    """
    if model == None:
        # Instanciando a regressão linear
        model = LinearRegression()
    
    # Obtendo as features e alvos a partir dos dados
    train_target = train_df_features['MaxTemp'].values
    train_data = train_df_features.drop(['MaxTemp', 'Date'], axis=1).values
    
    valid_target = valid_df_features['MaxTemp'].values
    valid_data = valid_df_features.drop(['MaxTemp', 'Date'], axis=1).values
    
    # Treinando o modelo
    model.fit(train_data, train_target)
    
    # Computando as predições e erro de validação
    predictions_train = model.predict(train_data)
    predictions_valid = model.predict(valid_data)
    validation_mse = mean_squared_error(valid_target, predictions_valid)
    
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.plot(train_df['Date'], train_df['MaxTemp'], color='b', marker='.', linestyle='', markersize=2, alpha=0.4)
    ax.plot(valid_df['Date'], valid_df['MaxTemp'], color='r', marker='.', linestyle='', markersize=2, alpha=0.4)
    
    ax.plot(train_df['Date'], predictions_train, color='b', label='Treino')
    ax.plot(valid_df['Date'], predictions_valid, color='r', label='Validação')
    ax.legend()
    
    ax.set_title('Predições de temperatura em função do tempo')
    
    plt.show()
    
    print("MSE (Erro quadrático médio): ", validation_mse)

## 3 - Extraindo features

In [ ]:
# Copiando os conjuntos de treino e validação
train_1 = train_df.copy()
valid_1 = valid_df.copy()

# Extraindo o mês
train_1['mes'] = train_1['Date'].dt.month
valid_1['mes'] = valid_1['Date'].dt.month
    
mostrar_predicoes(train_1, valid_1)

Gráficos semelhantes ao que foi criado acima serão utilizados nesse notebook para demonstrar como as predições se ajustam às features fornecidas.
Nele, os pontos representam os dados reais do clima de Sydney, enquanto as linhas contínuas representam as predições obtidas pela regressão linear.

Nessa primeira visualização, notamos que o gráfico obtido está muito distante dos dados reais, afinal, fornecemos apenas uma feature (mês) para um modelo de regressão linear simples. Para tentar melhorar isso, vamos extrair novas informações da data.

In [ ]:
# Copiando dos conjuntos anteriores
train_2 = train_1.copy()
valid_2 = valid_1.copy()

# Extraindo o dia
train_2['dia'] = train_2['Date'].dt.day
valid_2['dia'] = valid_2['Date'].dt.day

# Extraindo o ano
train_2['ano'] = train_2['Date'].dt.day
valid_2['ano'] = valid_2['Date'].dt.day

mostrar_predicoes(train_2, valid_2)

Percebe-se que o modelo ficou mais complexo, mas ainda não obtemos melhoria na performance do modelo (na verdade o erro aumentou). Logo, temos que fazer algum tratamento adicional nesses dados. Uma opção é aplicar one-hot encoding nas features, mesmo com elas sendo numéricas.

In [ ]:
# Copiando dos conjuntos só com o mês
train_3 = train_1.copy()
valid_3 = valid_1.copy()

# Aplicando one-hot encoding
train_3 = pd.get_dummies(train_3, columns=['mes'])
valid_3 = pd.get_dummies(valid_3, columns=['mes'])
train_3.head()

In [ ]:
# Predições usando apenas o mês
mostrar_predicoes(train_3, valid_3)

Nesse caso, já foi possível observar uma melhoria significante no desempenho do modelo. Isso acontece, porque ele pode tratar cada mês separadamente durante a regressão linear. Dessa forma, cada mês está devolvendo um valor diferente, enquanto o modelo anterior fazia com que a predição fosse o produto do número do mês por um valor somado a uma constante (linear).

- Sem one-hot encoding: MaxTemp = c * mês + constante.
- Com one-hot encoding: MaxTemp = c1 * é_janeiro + c2 * é_fevereiro + c3 * é_março + ... + constante

In [ ]:
# Copiando dos conjuntos anteriores
train_3 = train_2.copy()
valid_3 = valid_2.copy()

# Aplicando one-hot encoding
train_3 = pd.get_dummies(train_3, columns=['dia', 'mes', 'ano'])
valid_3 = pd.get_dummies(valid_3, columns=['dia', 'mes', 'ano'])
train_3.head()

# Mostrando as predicoes usando dia, mês e ano
mostrar_predicoes(train_3, valid_3)

Novamente, o modelo se tornou mais complexo, mas o erro aumentou. Isso foi um caso de overfitting, que pode ser resolvido selecionando melhor as features utilizadas ou usando outros métodos, como regularização.

In [ ]:
# Realizando uma regressão linear com regularização (Regressão de Ridge)
ridge_model = Ridge()
mostrar_predicoes(train_3, valid_3, ridge_model)

No modelo acima, usamos a regressão de Ridge, que é basicamente uma regressão linear simples com regularização L2. Observamos uma redução na complexidade do modelo, reduzindo o ruído causado pelos dias, com isso também obtemos um erro menor que as tentativas anteriores.

Leia mais sobre isso nesse link: https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/

## 4 - Features polinomiais

Uma desvantagem da regressão linear é a fato dela só extrair relações lineares das features fornecidas, mas isso pode ser contornado adicionando features polinomiais ao modelo (transformando ele em uma regressão polinomial), com isso, conseguimos extrair relações mais complexas dos mesmos dados. 

In [ ]:
train_4 = train_2.copy()
valid_4 = valid_2.copy()

# Instanciando o gerador de features do sklearn
poly = PolynomialFeatures(degree=2, include_bias=False)

# Adicionando as features geradas conjunto de treino e validação
train_4 = train_4.join(pd.DataFrame(poly.fit_transform(train_4[['dia', 'mes', 'ano']])))
valid_4 = valid_4.join(pd.DataFrame(poly.fit_transform(valid_4[['dia', 'mes', 'ano']])))

# Removendo colunas que ficaram duplicadas
train_4.drop(['dia', 'mes', 'ano'], axis=1, inplace=True)
valid_4.drop(['dia', 'mes', 'ano'], axis=1, inplace=True)

# Observe que o sklearn opera utilizando o numpy, logo, os valores gerados pelo fit_transform
# serão arrays do numpy. Para contornar isso, é possível usar pd.DataFrame() pra gerar um novo
# DataFrame a partir desse array, mas ele não possuirá nomes nas colunas, apenas números sequenciais
train_4.head()

In [ ]:
# Mostrando as predições geradas pela regressão polinomial
mostrar_predicoes(train_4, valid_4)

Uma observação importante é que não é faz sentido utilizar one-hot encoding seguido de geração de features polinomiais, pois todos os valores serão 0 ou 1, e esses números elevados a qualquer potências são eles mesmos.

Outra questão a prestar atenção na regressão polinomial é o grau da mesma, o gráfico acima foi feito com uma de grau 2. Podemos usar graus maiores para gerar modelos mais complexos, mas números muito altos também podem causar overfitting.

In [ ]:
train_4 = train_2.copy()
valid_4 = valid_2.copy()

# Instanciando o gerador de features do sklearn
poly = PolynomialFeatures(degree=4, include_bias=False)

# Adicionando as features geradas conjunto de treino e validacao
train_4 = train_4.join(pd.DataFrame(poly.fit_transform(train_4[['dia', 'mes', 'ano']])))
valid_4 = valid_4.join(pd.DataFrame(poly.fit_transform(valid_4[['dia', 'mes', 'ano']])))

train_4.drop(['dia', 'mes', 'ano'], axis=1, inplace=True)
valid_4.drop(['dia', 'mes', 'ano'], axis=1, inplace=True)

# Mostrando as predições geradas pela regressão polinomial
mostrar_predicoes(train_4, valid_4)

Utilizando features polinomiais de grau 4 obtemos o menor erro até agora, mas ainda podemos tentar aplicar regularização, por exemplo com Ridge e Lasso.

In [ ]:
# Regressão de Ridge com features polinomiais
ridge_model = Ridge()
mostrar_predicoes(train_4, valid_4, ridge_model)

In [ ]:
# Regressão de Lasso com features polinomiais
lasso_model = Lasso()
mostrar_predicoes(train_4, valid_4, lasso_model)

Com isso, observamos que nem sempre a regularização proporciona um desempenho melhor para o modelo.

## Conclusão

A partir dessa análise, é claro como extração de features e seu tratamento adequado melhoram consideravelmente o desempenho de um modelo. Também vale a pena destacar que modelos diferentes funcionam melhor com outros tipos de features (por exemplo modelos baseados em árvores).

Em casos de dados com séries temporais, extrair informações da data fornece ao modelo um comportamento em relação a periodicidade, como foi o caso desse notebook, afinal a temperatura segue um padrão com o decorrer dos meses. O mesmo poderia ser aplicado, por exemplo, às vendas de uma loja. Nesse caso, existiriam outras informações úteis, como saber se o dia é um feriado.

É importante ressaltar que tudo que foi apresentado é apenas uma parte pequena do campo da engenharia de features. Cada tipo de dado possui múltiplos tratamentos e codificações possíveis, com resultados diferentes. Porém, espero que esse material tenha te ajudado a compreender mais sobre esse campo tão amplo.